# Convolutional Neural Networks (Conv2d)

## Импорт библиотек

In [2]:
import torch
import torch.nn as nn

### Расчет внутренних параметров для модели

**Универсальная формула размера выхода сверточного слоя:**

$$
H_{out} = \left\lfloor \frac{H_{in} + 2P - K}{S} \right\rfloor + 1
$$

$$
W_{out} = \left\lfloor \frac{W_{in} + 2P - K}{S} \right\rfloor + 1
$$

Где:
- $H_{in}, W_{in}$ — высота и ширина входа,
- $K$ — размер ядра (kernel),
- $S$ — шаг (stride),
- $P$ — дополнение (padding),
- $\lfloor \cdot \rfloor$ — округление вниз.

**Формула для квадратных карт и ядер:**

$$
O = \left\lfloor \frac{I + 2P - K}{S} \right\rfloor + 1
$$

$(O = H_{out} = W_{out}, \quad I = H_{in} = W_{in})$

In [13]:
import sympy as sp

In [14]:
I, K, S, P = sp.symbols('I K S P', integer=True, positive=True)
O = sp.floor((I + 2*P - K) / S) + 1

In [15]:
sp.init_printing()
print("Формула для расчета размера выхода:")
O

Формула для расчета размера выхода:


import sympy as sp

# Определяем символы
I, K, S, P = sp.symbols('I K S P', integer=True, positive=True)
# Формула
O = sp.floor((I + 2*P - K) / S) + 1

# Красиво выводим
sp.init_printing()
print("Формула для расчета размера выхода:")
O

**Стандартные комбинации параметров для построения CNN**
    
    kernel_size = (3,3), stride = (1,1), padding = (0, 0)
    H_out = 
    W_out = 

    kernel_size = (3,3), stride = (1,1), padding = (1, 1)
    H_out = 
    W_out =

    kernel_size = (3,3), stride = (2,2), padding = (0, 0)
    H_out = 
    W_out =

### Первый способ создания модели

In [6]:
# модель, на вход котрой можно подать тензор любого размера, так как H и W не указаны, но их нужно контролировать

model = nn.Sequential(
            nn.Conv2d(3, 32, (3, 3)),   # inp => (batch_size, 3, 28, 28), out => (batch_size, 32, 26, 26)
            nn.ReLU(),
            nn.Conv2d(32, 64, (3, 3))   # inp => (batch_size, 32, 26, 26), out => (batch_size, 64, 24, 24)
        )

model

Sequential(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
)

In [8]:
input = torch.rand([16, 3, 28, 28], dtype = torch.float32)

out = model(input)
out.shape

torch.Size([16, 64, 24, 24])

### Второй способ создания модели

In [16]:
# модель, на вход котрой можно подать тензор любого размера, так как H и W не указаны, но их нужно контролировать

model = nn.Sequential()
model.add_module('layer_1', nn.Conv2d(3, 32, (3, 3)))   # inp => (batch_size, 3, 28, 28), out => (batch_size, 32, 26, 26)
model.add_module('ReLU', nn.ReLU())
model.add_module('layer_2', nn.Conv2d(32, 64, (3, 3)))   # inp => (batch_size, 32, 26, 26), out => (batch_size, 64, 24, 24)

model

Sequential(
  (layer_1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (ReLU): ReLU()
  (layer_2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
)

In [17]:
input = torch.rand([16, 3, 28, 28], dtype = torch.float32)

out = model(input)
out.shape

torch.Size([16, 64, 24, 24])

In [18]:
model.state_dict()

OrderedDict([('layer_1.weight',
  tensor([[[[-4.3474e-02,  1.3758e-01,  1.1815e-01],
            [ 4.6154e-02,  8.4222e-02, -5.3631e-02],
            [ 7.2689e-02,  1.8138e-01,  3.5662e-02]],
  
           [[-1.6521e-01, -1.2552e-01, -1.1840e-02],
            [ 1.5659e-01, -1.6170e-01, -1.4722e-01],
            [-1.7350e-01,  9.8617e-02, -1.7121e-01]],
  
           [[ 6.4164e-02,  1.5028e-01, -1.2906e-01],
            [-8.8608e-02, -1.6419e-01,  1.4358e-01],
            [ 2.7165e-02, -3.2719e-02, -2.4049e-02]]],
  
  
          [[[ 1.5683e-01, -3.1956e-02, -1.4567e-01],
            [-1.8770e-02, -8.9790e-02, -4.9384e-02],
            [-1.5656e-01, -1.1012e-01, -3.9929e-02]],
  
           [[-8.6451e-02, -1.4715e-01,  1.2724e-01],
            [ 1.5100e-01, -5.6248e-02, -6.9121e-02],
            [ 9.3976e-02, -7.1432e-02, -8.2556e-02]],
  
           [[-1.8676e-02, -1.0330e-01, -1.1004e-02],
            [ 4.9717e-02, -1.4982e-01, -1.1682e-01],
            [ 1.2476e-01,  4.1539e-02,  8.2

In [19]:
model.state_dict()['layer_1.weight'].shape

torch.Size([32, 3, 3, 3])